In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import winsound
os.environ["OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS"] = "0"
import cv2

from hand_task import HAND_TASK
from cell_phone_task import CELL_PHONE_TASK
from head_task import HEAD_DIRECTION_TASK
from iris_task import IRIS_TASK


iris_task = IRIS_TASK()
cell_phone_task = CELL_PHONE_TASK()
head_dir_task  = HEAD_DIRECTION_TASK()
hand_task = HAND_TASK()
tasks = [iris_task.process, cell_phone_task.process, head_dir_task.process, hand_task.process]

In [6]:
class WinSoundController:
    def __init__(self, wav_path: str):
        self.wav_path = wav_path
        self._on = False
    def set_on(self, on: bool):
        if on and not self._on:
            winsound.PlaySound(self.wav_path,
                               winsound.SND_FILENAME | winsound.SND_ASYNC | winsound.SND_LOOP)
            self._on = True
        elif not on and self._on:
            winsound.PlaySound(None, winsound.SND_PURGE)
            self._on = False


def add_label(res):
    label = res[1]
    color = (0,0,255) if res[0] else (255,0,0)  # keep your colors
    cv2.putText(res[2], label, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return res[2]

In [9]:
cap = cv2.VideoCapture(1)
cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)


# Use a WAV file for reliable start/stop control
sound = WinSoundController('alert.wav')
verbose = True

with ThreadPoolExecutor(max_workers=len(tasks)) as pool:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        futures = [pool.submit(func, frame.copy()) for func in tasks]
        results = [f.result() for f in futures]

        if verbose:
            concat_images = [add_label(res) for res in results]
            result_image = np.concatenate(
                (
                    np.concatenate((concat_images[0], concat_images[1]), axis=1),
                    np.concatenate((concat_images[2], concat_images[3]), axis=1)
                ), axis=0
            )
            cv2.imshow('result', result_image)

            # --- SOUND LOGIC: play once while any alert is True; stop when all False ---
            bad_flag = any(res[0] for res in results)
            sound.set_on(bad_flag)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
# ensure sound is stopped on exit
sound.set_on(False)
cv2.destroyAllWindows()
cap.release()

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore